In [26]:
# Setup
import pandas as pd
from xgboost import XGBClassifier
from typing import Union, Dict, List
from sklearn.ensemble import RandomForestClassifier

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
alldata = pd.concat([train_df.drop(['Survived'], axis=1), test_df])

In [21]:
# Missing values

with_na = ['Age', 'SibSp', 'Cabin']

alldata['Age'] = alldata['Age'].fillna(alldata['Age'].mean())#Drop cabin as it has too many missing values
alldata['Embarked'].fillna('S', inplace=True)
#Drop ticket as it's irrelavant
alldata= alldata.drop(['Ticket', 'Cabin'],axis=1)
alldata.isna().sum()
alldata['Fare']=alldata['Fare'].fillna(alldata['Fare'].mean())

In [22]:
def extract_titles(dataframe:pd.DataFrame)->Dict[str, int]:
    title_to_count = {}
    for fullname in dataframe['Name']:
        fn = fullname.split(',')
        surname = fn[0]
        given_name_and_title = fn[1]
        split_given_name_and_title = given_name_and_title.split()
        if split_given_name_and_title[0] in title_to_count:
            title_to_count[split_given_name_and_title[0]] +=1
        else:
            title_to_count[split_given_name_and_title[0]] = 1
    return title_to_count
titles_train= extract_titles(alldata)

def title_col(dataframe:pd.DataFrame) -> None:
    """
    Create a col for title
    :param dataframe:
    :return:
    """
    titles=[]
    for fullname in dataframe['Name']:
        fn= fullname.split(',')
        surname=fn[0]
        given_name_t=fn[1]
        t=given_name_t.split()
        titles.append(t[0])
    dataframe['Titles']=titles
title_col(alldata)
alldata= alldata.drop(['Name'], axis=1)

In [28]:
categorical_features = ['Pclass', 'Titles', 'Embarked', 'Sex']

# Encode the features and transform train and test
alldata = pd.get_dummies(alldata)

In [29]:
y = train_df['Survived']
trainX = alldata.head(891)
testX = alldata.tail(418)
model = XGBClassifier()
model.fit(trainX, y)
predictions = model.predict(testX)
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('xgbnew.csv', index=False)



C:\Users\warre\anaconda3\envs\Titanic\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:24:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
